In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow as tf
import transformers
from transformers import TFAutoModel, AutoTokenizer

In [ ]:
raw = pd.read_csv('/content/drive/MyDrive/60/Suicide_Detection.csv')

In [ ]:
df = raw.sample(1000 , random_state=29)
df= df.drop('Unnamed: 0',axis=1)
df['class'].replace({'suicide' : 1 , 'non-suicide' : 0}, inplace=True)
df

,text,class
213108,Nobody cares...Got to see / talk to my mom for...,1
214134,Lost and confusedSo this whole bout of depress...,1
136003,My life is falling apart and I don't know want...,1
192568,Mental BarrierHonestly I've stopped trying to ...,1
198908,I went ghost to focus on myself for a week and...,0
...,...,...
170224,I think I did an oopsie I posted here before s...,0
15887,A serious questionCan someone go to jail if a ...,1
111347,so dang tiredI have a neurological disorder th...,1
207239,Here we goGoing to smoke the rest of my Beelze...,1


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
encoded = tokenizer(df['text'].tolist(), truncation=True, padding=True)
encoded_dataset = tf.data.Dataset.from_tensor_slices((encoded, df['class'].tolist()))

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
BATCH_SIZE = 2

def order(inp, label):

    return {
        'input_ids': inp['input_ids'],
        'attention_mask': inp['attention_mask'],
        'token_type_ids': inp['token_type_ids']
    }, label



train_size = int(0.8 * len(encoded_dataset))
val_size = int(0.1 * len(encoded_dataset))

train_dataset = encoded_dataset.take(train_size).map(order)
val_dataset = encoded_dataset.skip(train_size).take(val_size).map(order)
test_dataset = encoded_dataset.skip(train_size + val_size).map(order)


train_dataset = train_dataset.map(order)
val_dataset = val_dataset.map(order)
test_dataset = test_dataset.map(order)


train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
val_dataset = val_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
text, label = next(iter(train_dataset))
print(text, '\n\n', label)

{'input_ids': <tf.Tensor: shape=(2, 512), dtype=int32, numpy=
array([[ 101, 3087, 2288, ...,    0,    0,    0],
       [ 101, 4299, 1045, ...,    0,    0,    0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 512), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 512), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>} 

 tf.Tensor([0 1], shape=(2,), dtype=int32)


In [ ]:
bert_model = TFAutoModel.from_pretrained("bert-base-uncased")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
class BERTForClassification(tf.keras.Model):

    def __init__(self, bert_model):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'bert_model': self.bert,
            'num_classes': self.fc.units,
        })
        return config

In [ ]:
model = BERTForClassification(bert_model)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)
history = model.fit(
    train_dataset,
    epochs=2,
    validation_data=val_dataset
)

Epoch 1/2
400/400 [==============================] - 5883s 15s/step - loss: 0.4167 - accuracy: 0.8150 - val_loss: 0.2296 - val_accuracy: 0.9100
Epoch 2/2
400/400 [==============================] - 5602s 14s/step - loss: 0.1797 - accuracy: 0.9413 - val_loss: 0.3208 - val_accuracy: 0.8500
